# Dataset Transform - excluding Household Type

In this notebook we are going to transform the data and vcreate a new (csv or parquet?) file that contains the total emissions per type of mobility  (`emissions_X`) and the development of choices (`stock_X`), where `X` can be any of the following types of mobility:
- `C`: combustion car
- `E`: electric car
- `N`: non-motorized.
- `P`: public transport.
- `S`: shared mobility.
The idea is to retrive tables like the following (I am actually thinking of making two csv files, one with the emissions and the other with the stock development):

| Step | reID | VarName | Value (total of regions) |
| --- | --- | --- | --- |
| 0 | 2335 | `emissions_C` | $\sum_{i=1}^{11}Chh_i$ |
| 0 | 2335 | `emissions_E` | $\sum_{i=1}^{11}Ehh_i$ |
| 0 | 2335 | `emissions_N` | $\sum_{i=1}^{11}Nhh_i$  |
| 0 | 2335 | `emissions_P` | $\sum_{i=1}^{11}Phh_i$  |
| 0 | 2335 | `emissions_S` | $\sum_{i=1}^{11}Shh_i$  |

There are a total of 181 steps (from 0 to 180), and for each mobility choice (there are 5 mobility choices), there are two variables regarding emissions and stock development, and there are 16 regions. Thus, the total number of rows is
$$\text{number of rows: }16\times 181\times 2\times 5=28960$$
Right now, each .csv files has **324352** rows.

## Roadmap

These are the ideas to make this iteratively:

1. Find a way in which I can get a list of the names of the files so that later on, we can iterate through them.
2. For each file, create a dataframe, and then
3. Transform the dataframe so that we ignore (for now) the types of household.

I even propose to start wsith a random file, do point 2, and once that is ready, we proceed to do it for all files.

In [1]:
import numpy as np
import pandas as pd

import datetime as dt
import time

import os
from os import listdir
from os.path import isfile,join
import matplotlib.pyplot as plt

In [2]:
PATH = '/home/moni/Documents/motmo/timeSeries_files/' # original data
PATH2 = '/home/moni/Documents/motmo/data_without_hhID/' # folder in which we will store transformed data

### For the record

The file we chose at random is the choice in which there is "Investment in public transport" (`SP`) and in "Electric vehicel subsidy" (`SE`), and there is the *event*  (`CO`) in which the gas price is higher.

In [5]:
hh_df = pd.read_csv(PATH + 'timeSeries_CH0SP0SE0WE0BP0RE0CO0DI0WO1CS1.csv')
# hh_df = hh_df[hh_df.varName != 'elDem']
# hh_df = hh_df[hh_df.varName != 'nStat']
# hh_df.head(15)

### Ignoring Household Type
We will ignore the household type and will sum all over the timesteps, regardless of `hhID`. For that, we create a function called `del_hhID_from_df(hh_df)` that takes as input the original dataframe and returns the desired one.


In [3]:
def del_hhID_from_df(hh_df):
    hh_df = hh_df[hh_df.varName != 'elDem']
    hh_df = hh_df[hh_df.varName != 'nStat']
    hh_df = hh_df.groupby(['step','reID','varName']).sum().reset_index()
    del hh_df['hhID']
    hh_df.head(12)
    hh_df = hh_df.pivot_table(values='value', index=['step','reID'], columns='varName', fill_value=0, aggfunc='sum').rename_axis(None, axis=1).reset_index()
    return hh_df

In [7]:
h_df = del_hhID_from_df(hh_df)
h_df

,step,reID,emissions_C,emissions_E,emissions_N,emissions_P,emissions_S,stock_C,stock_E,stock_N,stock_P,stock_S
0,0,942,4.768518e+08,1.128790e+05,245.377178,2.748710e+07,2.655663e+05,38051,7,3131,5466,20
1,0,1515,2.685138e+09,1.245850e+06,1659.179759,2.470828e+08,2.190017e+06,222222,82,21337,49462,174
2,0,1516,1.683078e+09,2.166748e+05,1068.788266,1.088996e+08,8.323659e+05,137830,15,13924,21939,70
3,0,1517,9.451544e+08,4.605658e+05,564.617520,6.933001e+07,7.832123e+05,78182,27,7218,13920,58
4,0,1518,8.412279e+07,0.000000e+00,34.442713,6.044010e+06,4.971810e+04,7385,0,467,1392,5
...,...,...,...,...,...,...,...,...,...,...,...,...
2891,180,2335,3.938841e+08,1.628617e+07,72.587368,8.948445e+06,9.063947e+05,34635,3204,866,2526,75
2892,180,2336,6.738755e+08,2.888294e+07,149.953848,1.384856e+07,2.254944e+06,57012,5787,1820,4124,190
2893,180,3312,2.920709e+08,1.049765e+07,59.425671,5.987746e+06,3.747730e+05,24272,1963,666,1698,36
2894,180,3562,4.689066e+08,2.084883e+07,178.903118,1.188505e+07,6.335879e+07,34284,3589,2009,2908,5983


In [30]:
hh_df.pivot_table(values='value', index=['step','reID'], columns='varName', fill_value=0, aggfunc='sum').rename_axis(None, axis=1).reset_index()

,step,reID,emissions_C,emissions_E,emissions_N,emissions_P,emissions_S,stock_C,stock_E,stock_N,stock_P,stock_S
0,0,942,4.755727e+08,1.374278e+05,246.140453,2.784241e+07,5.124479e+05,38017,10,3138,5478,32
1,0,1515,2.683702e+09,1.304724e+06,1666.876390,2.471230e+08,2.177926e+06,222052,77,21305,49671,172
2,0,1516,1.679456e+09,4.629271e+05,1085.205175,1.090998e+08,9.204325e+05,137599,24,14124,21941,90
3,0,1517,9.461965e+08,3.718691e+05,568.895558,6.855280e+07,6.988432e+05,78330,22,7155,13843,55
4,0,1518,8.350812e+07,4.289803e+03,35.642918,6.222572e+06,3.271747e+04,7371,1,482,1392,3
...,...,...,...,...,...,...,...,...,...,...,...,...
2891,180,2335,3.586460e+08,6.963521e+06,40.572440,2.750924e+07,6.798272e+05,32288,1251,446,7272,49
2892,180,2336,6.203866e+08,1.677639e+07,88.334120,4.130009e+07,8.577773e+05,53581,3091,1034,11174,53
2893,180,3312,2.668280e+08,4.439659e+06,38.190878,1.874227e+07,3.062764e+05,22657,779,390,4788,21
2894,180,3562,4.664780e+08,1.567774e+07,89.412193,3.822309e+07,1.511763e+07,35094,2582,992,8730,1375


In [9]:
def from_df_to_csv(df,file_name):
    df.to_csv(PATH2 + file_name)# saves in the new location

In [11]:
from_df_to_csv(h_df,"timeSeries_CH0SP0SE0WE0BP0RE0CO0DI0WO1CS1.csv")

## Retreiving all file names
We want a list with all file names of each of the 539 scenarios

In [9]:
def list_file_names():
    file_names = [f for f in listdir(PATH) if isfile(join(PATH, f))]
    file_names = [name.replace(".csv", "") for name in file_names]
    return file_names

In [10]:
file_n = list_file_names().copy()

### Saving all files
This function takes as input the converted dataframe.

CAREFUL!!! only run this once, since it takes some minutes and if you do it and the files are already there, there might be overlapping or errors.

In [7]:
def from_csv_to_df(file_name):
    df = pd.read_csv(PATH + file_name)
    return df    

In [20]:
def saving_all_files(file_name_list):
    n = len(file_name_list)
    
    for i in range(0,n):
        df = pd.read_csv(PATH + file_name_list[i] +'.csv')
        df = del_hhID_from_df(df)
        from_df_to_csv(df,file_name_list[i]+'.csv')

### CAREFUL
Only run (uncomment) this line if you have not saved the new dataset that excludes household types!!!

In [22]:
saving_all_files(file_n)

### Let's try something
I will try to save all these .csv files into one parquet file

IDEA FOR NEXT TIME: from the list that contains all names, create a new list that deletes the ".csv" extension at the end, and also the "TimeSeries" that each file name starts with, and then try to do what has been donde here to save it all in one parquet file: https://stackoverflow.com/questions/63509110/convert-multiple-csvs-to-single-partitioned-parquet-dataset

The code below is not complete, do not run!!!

### Cleaning list names

I want to create a parquet file such that it adds the code of the scenario for each entry
NOTE: THIS PART IS UNDER CONSTRUCTION, I WILL DO IT LATER!!!

In [8]:
def clean_list(list_files):
    n_list = [name.replace(".csv", "") for name in list_files]
    n_list = [name.replace("timeSeries_", "") for name in n_list]
    return n_list


In [ ]:
file_names = clean_list(file_n).copy() # we need a new list!!!
for f in file_n: # HERE I do want the full name, since we are reading this
    df = pd.read_csv(PATH2 + f)
    df = del_hhID_from_df(df).drop(columns=['value'])
    df.to_parquet(f'all_years.pq/YEAR={year}')
fastparquet.writer.merge([f'all_years.pq/YEAR={y}' for y in years])

df_all = pd.read_parquet('all_years.pq')